<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/Feature_Factory_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
conda-forge/linux-64  ⣾  [+] 0

In [ ]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176723 sha256=0352c466d72783732b78fe756bdb80a5b01e5fb6738dd16db05829f8082f01fa
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import os

from sklearn.model_selection import KFold, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import load_model

from rdkit import Chem
from rdkit.Chem import PandasTools, AllChem, Draw, ChemicalFeatures
from rdkit.Chem import Descriptors
from rdkit import RDConfig
from rdkit import DataStructs

from mordred import Calculator, descriptors


In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

#Descriptor 함수 정의

In [ ]:
#1 RDkit에서 제공하는 Descriptor
def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

#2 atomic environment/ 각 원자의 주변원자, 혼성오비탈, ring에 포함되어 있는 원자인지 / 각 atomic environment를 피쳐로 개수 count
def extract_fragments(smiles, hybridization_type=None, ring=False):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        hybridization = atom.GetHybridization()
        if (hybridization_type is None or hybridization == hybridization_type) and ring and atom.IsInRing():
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

        if (hybridization_type is None or hybridization == hybridization_type) and not ring and not atom.IsInRing():
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

def count_fragments(full, column_prefix, hybridization_type=None, ring=False):
    fragment_counts = {}
    for index, row in full.iterrows():
        smiles = row['SMILES']
        fragments = extract_fragments(smiles, hybridization_type, ring)

        fragment_count = {}
        for idx, fragment in enumerate(fragments):
            fragment_smiles = Chem.MolToSmiles(fragment)
            if fragment_smiles in fragment_count:
                fragment_count[fragment_smiles] += 1
            else:
                fragment_count[fragment_smiles] = 1

        fragment_counts[index] = fragment_count

    result_data = []
    for index, counts in fragment_counts.items():
        counts['SMILES'] = full.loc[index, 'SMILES']
        result_data.append(counts)

    result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
    result_df.columns = [f'{column_prefix}_{column}' for column in result_df.columns]

    return result_df

#3 입체이성질체유무, 전체에서 aromatic 원자 비율, formal charge
def calculate_molecular_properties(full):
    def calculate_stereocenters(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
        return len(chiral_centers)

    def calculate_aromatic_proportion(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        aromatic_atoms = [atom for atom in mol.GetAtoms() if atom.GetIsAromatic()]
        total_atoms = mol.GetNumAtoms()
        if total_atoms == 0:
            return None
        return len(aromatic_atoms) / total_atoms

    def calculate_formal_charge(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return Chem.rdmolops.GetFormalCharge(mol)

    properties_data = []

    for index, row in full.iterrows():
        smiles = row['SMILES']
        stereocenters = calculate_stereocenters(smiles)
        proportion = calculate_aromatic_proportion(smiles)
        charge = calculate_formal_charge(smiles)

        if stereocenters is not None and proportion is not None and charge is not None:
            properties_data.append({
                "stereocenters": stereocenters,
                "aromatic_proportion": proportion,
                "formal_charge": charge
            })

    properties_df = pd.DataFrame(properties_data)

    return properties_df

#데이터 가공

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

# #ChEMBL data
# MLM = pd.read_csv("/content/drive/MyDrive/chembl_MLM_logD.csv")
# HLM = pd.read_csv("/content/drive/MyDrive/chembl_HLM_logD.csv")

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

# full = pd.concat([train, test, MLM, HLM], axis = 0).reset_index(drop=True)
full = train[:500]
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

<ipython-input-17-99ccf3c286dd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)


In [ ]:
filtered_full = full

In [ ]:
# Mordred Descriptor
calc = Calculator(descriptors, ignore_3D=True)
mord_desc_df = calc.pandas([Chem.MolFromSmiles(x) for x in filtered_full.SMILES])
mord_desc_df.columns = ['mord_'+ column for column in mord_desc_df.columns]
mord_desc_df.drop(columns=mord_desc_df.select_dtypes(include=['object']).columns, inplace=True)
bool_columns = mord_desc_df.select_dtypes(include=['bool']).columns
mord_desc_df[bool_columns] = mord_desc_df[bool_columns].astype(int)

print(mord_desc_df.head())

 50%|████▉     | 249/500 [01:19<03:14,  1.29it/s]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 500/500 [02:28<00:00,  3.37it/s]


    mord_ABC  mord_ABCGG  mord_nAcid  mord_nBase  mord_SpAbs_A  mord_SpMax_A  \
0  21.379612   17.449011           0           0     35.689316      2.421199   
1  16.539255   14.049653           0           0     26.575899      2.426398   
2  17.475469   13.660693           2           1     29.802128      2.510668   
3  27.857311   20.034364           0           1     45.884166      2.532483   
4  15.722758   12.817176           0           0     26.308663      2.452930   

   mord_SpDiam_A  mord_SpAD_A  mord_SpMAD_A  mord_LogEE_A  ...  mord_SRW10  \
0       4.745523    35.689316      1.274618      4.249154  ...   10.081676   
1       4.757199    26.575899      1.265519      3.984419  ...    9.907828   
2       4.982923    29.802128      1.354642      4.049690  ...   10.144510   
3       4.973440    45.884166      1.310976      4.500758  ...   10.613467   
4       4.905860    26.308663      1.315433      3.935574  ...    9.978363   

   mord_TSRW10     mord_MW  mord_AMW  mord_WPath  

In [ ]:
allDescrs = [getMolDescriptors(m) for m in filtered_full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

In [ ]:
result_df_sp3 = count_fragments(filtered_full, 'ring_sp3', Chem.HybridizationType.SP3, ring=True)
result_df_non_ring_sp3 = count_fragments(filtered_full, 'non_ring_sp3', Chem.HybridizationType.SP3)
result_df_sp2 = count_fragments(filtered_full, 'ring_sp2', Chem.HybridizationType.SP2, ring=True)
result_df_non_ring_sp2 = count_fragments(filtered_full, 'non_ring_sp2', Chem.HybridizationType.SP2)
result_df_sp = count_fragments(filtered_full, 'sp', Chem.HybridizationType.SP)

prop_df = calculate_molecular_properties(filtered_full)

In [ ]:

fdefName = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')

factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

feature_counts = {}

def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    feats = factory.GetFeaturesForMol(mol)

    feature_count = {}
    for feat in feats:
        feat_type = feat.GetType()
        if feat_type in feature_count:
            feature_count[feat_type] += 1
        else:
            feature_count[feat_type] = 1

    return feature_count

for index, row in filtered_full.iterrows():
    smiles = row['SMILES']
    feature_count = extract_features(smiles)
    feature_counts[index] = feature_count


result_data = []
for index, counts in feature_counts.items():
    counts['SMILES'] = filtered_full.loc[index, 'SMILES']
    result_data.append(counts)

result_df_features = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')


result_df_features.columns = ['feature_' + column for column in result_df_features.columns]


print(result_df_features)


     feature_SingleAtomDonor  feature_SingleAtomAcceptor  feature_Arom5  \
0                        3.0                         5.0            2.0   
1                        1.0                         2.0            1.0   
2                        1.0                         4.0            1.0   
3                        1.0                         5.0            1.0   
4                        0.0                         3.0            0.0   
..                       ...                         ...            ...   
495                      3.0                         2.0            1.0   
496                      2.0                         4.0            0.0   
497                      5.0                         4.0            1.0   
498                      1.0                         2.0            0.0   
499                      2.0                         3.0            1.0   

     feature_Arom6  feature_ThreeWayAttach  feature_RH6_6  feature_BasicGroup  \
0              1.0

In [ ]:
Data = pd.concat([filtered_full, full_Descrs ,mord_desc_df,result_df_sp3, result_df_non_ring_sp3, result_df_sp2, result_df_non_ring_sp2, result_df_sp, prop_df, result_df_features], axis=1).drop(columns=['Molecule'])
Data["Molecular_PolarSurfaceArea"] = np.where(pd.isna(Data["Molecular_PolarSurfaceArea"]), Data["TPSA"], Data["Molecular_PolarSurfaceArea"])
Data["AlogP"] = np.where(pd.isna(Data["AlogP"]), Data["LogD"], Data["AlogP"])

In [ ]:
# 약물가능성(주로 막투과도, 용해도 관련) 평가요소인 lipinski rule of 5와 추가 평가요소

Data['beyond_Lipinski'] = 0

Data.loc[Data['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
Data.loc[Data['LogD'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[(Data['Num_H_Donors'] + Data['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
Data.loc[Data['TPSA'] >= 140, 'beyond_Lipinski'] += 1
Data.loc[Data['mord_FCSP3'] <= 0.3, 'beyond_Lipinski'] += 1
Data.loc[Data['mord_naRing'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['NOCount'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['NHOHCount'] >= 5, 'beyond_Lipinski'] += 1

In [ ]:
# 약물의 막투과도 관련 feature
Data['ACDNSA'] = Data['Num_H_Acceptors']*np.sqrt(Data['Num_H_Donors'])/Data['mord_LabuteASA']
Data['globe'] =  Data['mord_Radius']*Data['mord_Radius']/Data['mord_LabuteASA']
Data['LogD_Radius'] = Data['LogD']/(Data['mord_Radius']*Data['Molecular_Weight']) #구형성피쳐로 바꿔주기

In [ ]:
Data = pd.read_csv('/content/drive/MyDrive/metabolism_dacon/features/Data_27.39.csv')

In [ ]:
Data

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,...,heavy_atom_count,formal_charge,Molar Refractivity,VdW Volume,Flexibility,Electronegativity,Pharmacophore Features,rule_of_five_satisfactions,QPlogPo_w,beyond_Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.259,...,28,0,109.1199,28,8,188,16,0,3.87744,0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.172,...,21,0,86.8737,21,2,141,11,0,3.35474,0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.585,...,22,0,85.0270,22,3,139,13,0,1.20450,0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.475,...,35,0,136.0058,35,5,230,17,0,3.89356,0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.337,...,20,0,77.1825,20,1,126,12,0,2.81772,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5573,TEST_478,CCc1noc(CC)c1CC(=O)NCC1(CC)CCCCC1,NaN,NaN,4.207,306.443,2,1,7,4.207,...,22,0,87.7817,22,7,138,5,0,3.81860,0
5574,TEST_479,CC(=O)N1CCC2(CC1)OC(=O)C(C)=C2C(=O)N1CCN(C)CC1,NaN,NaN,-0.608,335.398,5,0,1,-1.736,...,24,0,87.0880,24,1,155,0,0,0.01480,0
5575,TEST_480,CC(C)NC(=O)CN1C(=O)c2ccccc2N2C(=O)c3ccccc3C12,NaN,NaN,1.792,349.383,3,1,3,1.792,...,26,0,96.7497,26,3,165,12,0,2.32600,1
5576,TEST_481,Cn1cc(Br)c(=O)c(NC(=O)c2ccc(O)cc2F)c1,NaN,NaN,0.790,341.132,3,2,2,0.423,...,20,0,75.4010,20,2,160,12,0,2.24480,1


#추가시킬 피쳐

In [ ]:
!pip install useful_rdkit_utils mols2grid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.4/295.4 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.6/806.6 kB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from rdkit import Chem
import mols2grid
import useful_rdkit_utils as uru
from tqdm.auto import tqdm
from itertools import chain

In [ ]:
df = Data[['id', 'SMILES']]

In [ ]:
ring_system_finder = uru.RingSystemFinder()

ring_counts = {}

def extract_ring_counts(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    feats = ring_system_finder.find_ring_systems(mol)

    feature_count = {}
    for feat in feats:
        if feat in feature_count:
            feature_count[feat] += 1
        else:
            feature_count[feat] = 1

    return feature_count

for index, row in Data.iterrows():
    smiles = row['SMILES']
    feature_count = extract_ring_counts(smiles)
    ring_counts[index] = feature_count


result_data = []
for index, counts in ring_counts.items():
    counts['SMILES'] = Data.loc[index, 'SMILES']
    result_data.append(counts)

ring_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')


ring_df.columns = ['RING_' + column for column in result_df_features.columns]


print(ring_df)

In [ ]:
from rdkit import Chem
import pandas as pd

def extract_radius2_atomenv(smiles, radius):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3 and atom.IsInRing() == False:
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom.GetIdx())
            submol = Chem.PathToSubmol(mol, env)
            fragments.append(submol)

    return fragments

radius = 1
fragment_counts = {}

# 데이터프레임의 각 행에 대해 반복
for index, row in Data.iterrows():
    smiles = row['SMILES']
    fragments = extract_radius2_atomenv(smiles, radius)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = Data.loc[index, 'SMILES']
    result_data.append(counts)

result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# 결과 데이터프레임 출력
print(result_df)


       CC  CCO  cCN   cC   Cc  CCC  CCN  cS(N)(=O)=O   CO   cF  ...  \
0     2.0  2.0  1.0  2.0  0.0  0.0  0.0          0.0  0.0  0.0  ...   
1     1.0  0.0  1.0  1.0  1.0  0.0  0.0          0.0  0.0  0.0  ...   
2     1.0  0.0  0.0  0.0  0.0  1.0  1.0          0.0  0.0  0.0  ...   
3     0.0  0.0  0.0  2.0  1.0  0.0  0.0          1.0  0.0  0.0  ...   
4     1.0  0.0  0.0  0.0  1.0  0.0  0.0          0.0  0.0  0.0  ...   
...   ...  ...  ...  ...  ...  ...  ...          ...  ...  ...  ...   
5573  3.0  0.0  0.0  0.0  0.0  1.0  1.0          0.0  0.0  0.0  ...   
5574  2.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  0.0  ...   
5575  2.0  0.0  0.0  0.0  0.0  0.0  1.0          0.0  0.0  0.0  ...   
5576  0.0  0.0  0.0  0.0  0.0  0.0  0.0          0.0  0.0  1.0  ...   
5577  2.0  0.0  0.0  0.0  0.0  1.0  1.0          0.0  0.0  0.0  ...   

      O=P([O-])(O)O  [O-]P   OP  C[Si]  CP(=O)(O)O  CC(P)P  c[Si](C)(C)C  \
0               0.0    0.0  0.0    0.0         0.0     0.0           0.

In [ ]:
from rdkit import Chem
import pandas as pd

def extract_radius2_atomenv(smiles, radius):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == False:
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom.GetIdx())
            submol = Chem.PathToSubmol(mol, env)
            fragments.append(submol)

    return fragments

radius = 1
fragment_counts = {}

# 데이터프레임의 각 행에 대해 반복
for index, row in Data.iterrows():
    smiles = row['SMILES']
    fragments = extract_radius2_atomenv(smiles, radius)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = Data.loc[index, 'SMILES']
    result_data.append(counts)

result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# 결과 데이터프레임 출력
print(result_df)

      COc  CNC  cC(N)=O  C=O  cOC  O=S  c=O  CN(C)c  CC(=O)O   CO  ...  \
0     1.0  1.0      1.0  1.0  1.0  0.0  0.0     0.0      0.0  0.0  ...   
1     0.0  0.0      0.0  1.0  0.0  0.0  0.0     0.0      0.0  0.0  ...   
2     0.0  0.0      0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0  ...   
3     0.0  0.0      0.0  0.0  0.0  2.0  0.0     0.0      0.0  0.0  ...   
4     0.0  0.0      1.0  1.0  0.0  0.0  0.0     0.0      0.0  0.0  ...   
...   ...  ...      ...  ...  ...  ...  ...     ...      ...  ...  ...   
5573  0.0  1.0      0.0  1.0  0.0  0.0  0.0     0.0      0.0  0.0  ...   
5574  0.0  0.0      0.0  3.0  0.0  0.0  0.0     0.0      0.0  0.0  ...   
5575  0.0  1.0      0.0  3.0  0.0  0.0  0.0     0.0      0.0  0.0  ...   
5576  0.0  0.0      1.0  1.0  0.0  0.0  1.0     0.0      0.0  0.0  ...   
5577  0.0  0.0      0.0  0.0  0.0  0.0  0.0     0.0      0.0  0.0  ...   

      CC(=N)N  C=NP  O=P  cOP  C=CCl  NC=O  cN(S)S  CN=[N+]  [N+]=[N-]  \
0         0.0   0.0  0.0  0.0    0.0 

In [ ]:
from rdkit import Chem
import pandas as pd

def extract_radius2_atomenv(smiles, radius):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP2 and atom.IsInRing() == True:
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom.GetIdx())
            submol = Chem.PathToSubmol(mol, env)
            fragments.append(submol)

    return fragments

radius = 2
fragment_counts = {}

# 데이터프레임의 각 행에 대해 반복
for index, row in Data.iterrows():
    smiles = row['SMILES']
    fragments = extract_radius2_atomenv(smiles, radius)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = Data.loc[index, 'SMILES']
    result_data.append(counts)

result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# 결과 데이터프레임 출력
print(result_df)

      ccc(OC)c(c)O  cccc(c)O  cccc(c)C  ccc(cc)CN  ccc([nH]n)C(N)=O  \
0              2.0       1.0       1.0        1.0               1.0   
1              0.0       0.0       0.0        0.0               0.0   
2              0.0       0.0       0.0        0.0               0.0   
3              0.0       0.0       2.0        0.0               0.0   
4              0.0       1.0       1.0        0.0               0.0   
...            ...       ...       ...        ...               ...   
5573           0.0       0.0       0.0        0.0               0.0   
5574           0.0       0.0       0.0        0.0               0.0   
5575           0.0       0.0       3.0        0.0               0.0   
5576           0.0       1.0       1.0        0.0               0.0   
5577           0.0       0.0       0.0        0.0               0.0   

      Cc([nH])cc(-c)n  ccc(n[nH])-c(c)s  csc(-c(c)n)c(C)n  cc(-c)sc(C)n  \
0                 1.0               1.0               1.0           1.0 

In [ ]:
# 각 컬럼마다 0이 아닌 값의 개수를 세는 코드
non_zero_counts = result_df.apply(lambda col: col[col != 0].count())

# 결과 출력
print(non_zero_counts.sort_values(ascending=False).head(1024))

cccc(c)C            2563
ccccc               2462
cccc(c)N            1094
cccc(c)O            1093
cccc(c)-c            928
                    ... 
cnc(CN)[nH]c          11
cnc(C(N)=O)c(c)C      11
cc(-c)c(OC)c(c)C      11
cnc([nH]c)SC          11
cc(C)nc(C)o           11
Length: 1024, dtype: int64


In [ ]:
from rdkit import Chem
import pandas as pd

def extract_radius2_atomenv(smiles, radius):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        if atom.GetHybridization() == Chem.HybridizationType.SP3 and atom.IsInRing() == True:
            env = Chem.FindAtomEnvironmentOfRadiusN(mol, radius, atom.GetIdx())
            submol = Chem.PathToSubmol(mol, env)
            fragments.append(submol)

    return fragments

radius = 1
fragment_counts = {}

# 데이터프레임의 각 행에 대해 반복
for index, row in Data.iterrows():
    smiles = row['SMILES']
    fragments = extract_radius2_atomenv(smiles, radius)

    fragment_count = {}
    for idx, fragment in enumerate(fragments):
        fragment_smiles = Chem.MolToSmiles(fragment)
        if fragment_smiles in fragment_count:
            fragment_count[fragment_smiles] += 1
        else:
            fragment_count[fragment_smiles] = 1

    fragment_counts[index] = fragment_count

# fragment_counts를 사용하여 새로운 데이터프레임 생성
result_data = []
for index, counts in fragment_counts.items():
    counts['SMILES'] = Data.loc[index, 'SMILES']
    result_data.append(counts)

result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# 결과 데이터프레임 출력
print(result_df)

      CCN  CC(C)C  CN(C)C  CN(C)S  CC(C)N  CCC  CC(C)O  CC(C)n  CNC  cCN  ...  \
0     0.0     0.0     0.0     0.0     0.0  0.0     0.0     0.0  0.0  0.0  ...   
1     1.0     1.0     0.0     0.0     0.0  0.0     0.0     0.0  0.0  0.0  ...   
2     4.0     0.0     1.0     0.0     0.0  0.0     0.0     0.0  0.0  0.0  ...   
3     4.0     0.0     1.0     1.0     1.0  5.0     0.0     0.0  0.0  0.0  ...   
4     1.0     0.0     0.0     0.0     0.0  0.0     1.0     0.0  0.0  0.0  ...   
...   ...     ...     ...     ...     ...  ...     ...     ...  ...  ...  ...   
5573  0.0     0.0     0.0     0.0     0.0  5.0     0.0     0.0  0.0  0.0  ...   
5574  6.0     0.0     1.0     0.0     0.0  2.0     0.0     0.0  0.0  0.0  ...   
5575  0.0     0.0     0.0     0.0     0.0  0.0     0.0     0.0  0.0  0.0  ...   
5576  0.0     0.0     0.0     0.0     0.0  0.0     0.0     0.0  0.0  0.0  ...   
5577  4.0     0.0     2.0     0.0     0.0  0.0     0.0     0.0  0.0  0.0  ...   

      NS(N)(=O)=O  cC(C)(F)

In [ ]:
from rdkit.Chem import AllChem

fp_list = []
for idx, smiles in enumerate(Data["SMILES"]):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetHashedMorganFingerprint(mol, 8, nBits=4096*2, useChirality=True, useBondTypes=True)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp])

dic = {}
for idx in range(4096*2):
  key = "fcfp8_" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(4096*2):
    key = "fcfp8_" + str(idx)
    dic[key].append(fp[idx])

FCFP_8_4096_2 = pd.DataFrame(dic)

In [ ]:
from rdkit.Chem import AllChem

fp_list = []
for idx, smiles in enumerate(Data["SMILES"]):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetHashedMorganFingerprint(mol, 4, nBits=2048, useChirality=True, useBondTypes=True, useFeatures=True)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp])

dic = {}
for idx in range(2048):
  key = "fcfp4_" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(2048):
    key = "fcfp4_" + str(idx)
    dic[key].append(fp[idx])

FCFP_4_2048 = pd.DataFrame(dic)

In [ ]:
from rdkit.Chem import AllChem

fp_list = []
for idx, smiles in enumerate(Data["SMILES"]):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096, useChirality=True, useBondTypes=True, useFeatures=True)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp])

dic = {}
for idx in range(2048):
  key = "fcfp6_" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(2048):
    key = "fcfp6_" + str(idx)
    dic[key].append(fp[idx])

FCFP_6_4096 = pd.DataFrame(dic)

In [ ]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys

# SMILES 데이터가 있는 열에서 SMILES를 읽어온다.
smiles_list = Data["SMILES"]

# MACCS fingerprint를 저장할 리스트를 생성한다.
maccs_fp_list = []

# SMILES를 이용하여 MACCS fingerprint를 계산한다.
for smiles in smiles_list:
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        maccs_fp = MACCSkeys.GenMACCSKeys(mol)
        maccs_fp_list.append(maccs_fp)

# MACCS fingerprint를 이진 리스트로 변환한다.
maccs_fp_bit_list = []
for maccs_fp in maccs_fp_list:
    maccs_fp_bit_list.append([int(x) for x in maccs_fp.ToBitString()])

# 이진 fingerprint를 데이터프레임으로 변환한다.
import pandas as pd

dic = {}
for idx in range(167):
    key = "MACCS_" + str(idx)
    dic[key] = []

for maccs_fp in maccs_fp_bit_list:
    for idx in range(167):
        key = "MACCS_" + str(idx)
        dic[key].append(maccs_fp[idx])

MACCS_keys = pd.DataFrame(dic)

By default, GetHashedMorganFingerprint() uses the "useFeatures=False" option to provide an implementation of the ECFP (Extended Connectivity Fingerprint) method as described in the "ECFP (Extended Connectivity Fingerprint) Method - 2000" section in this book.

In [ ]:
from rdkit.Chem import AllChem
fp_list = []
for idx, smiles in enumerate(Data["SMILES"]): # Smiles 열에서 반복.
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits = 4096)
    fp_list.append(fp)

import sys
fp_bit_list = []
for fp in fp_list:
  fp_bit_list.append([int(x) for x in fp.ToBitString()])

dic = {}
for idx in range(4096):
  key = "F" + str(idx)
  dic[key] = []

for fp in fp_bit_list:
  for idx in range(4096):
    key = "F" + str(idx)
    dic[key].append(fp[idx])

ECFP_fp_4096 = pd.DataFrame(dic)

In [ ]:
!pip install datamol
!pip install molfeat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.8/381.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 89.1 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.0/567.0 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.9/175.9 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.8/118.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.4/181.4 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import datamol as dm

from rdkit.Chem import SaltRemover
from molfeat.trans.fp import FPVecTransformer
from molfeat.trans.concat import FeatConcat
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

ETC_COLUMNS = ["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea"]
AVAILABLE_FPS = ['maccs', 'avalon', 'ecfp', 'fcfp', 'topological', 'atompair', 'rdkit', 'pattern', 'layered', 'secfp', 'erg', 'estate', 'avalon-count', 'rdkit-count', 'ecfp-count', 'fcfp-count', 'topological-count', 'atompair-count',
                 'cats2D', 'pharm2D', 'scaffoldkeys', 'skeys']
MODEL = "v9_2"
SEED = 42

dm.disable_rdkit_log()

def preprocess_mol(row):
    mol = dm.to_mol(row["SMILES"], ordered=True)
    # mol = dm.fix_mol(mol)
    # mol = dm.sanitize_mol(mol, sanifix=True, charge_neutral=False)
    # mol = dm.standardize_mol(
    #     mol,
    #     disconnect_metals=False,
    #     normalize=True,
    #     reionize=True,
    #     uncharge=False,
    #     stereo=True,
    # )

    #mol = SaltRemover.SaltRemover().StripMol(mol, dontRemoveEverything=True)
    row["Standard_Smiles"] = dm.to_smiles(mol)

    return row

def fill_na(df, imputer=None):
    if imputer is None:
        imputer = IterativeImputer(estimator=RandomForestRegressor(n_jobs=-1), random_state=SEED)
        df[ETC_COLUMNS] = imputer.fit_transform(df[ETC_COLUMNS].to_numpy())

        return pd.DataFrame(df), imputer
    else:
        df[ETC_COLUMNS] = imputer.transform(df[ETC_COLUMNS].to_numpy())

        return pd.DataFrame(df)

def extract_features(df):
    _df = df.apply(preprocess_mol, axis=1)

    fps = []
    for fp in AVAILABLE_FPS:
        fps.append(FPVecTransformer(fp, dtype=np.float64, n_jobs=-1))

    featurizer = FeatConcat(fps, dtype=np.float64)
    smiles = _df["Standard_Smiles"].to_list()
    descriptors = featurizer(smiles)

    etcs = _df[ETC_COLUMNS].to_numpy()

    return pd.DataFrame(np.concatenate([descriptors], axis=1))

exf_full = extract_features(Data)


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [ ]:
import datamol as dm
import pandas as pd

def calculate_molecular_properties_dm(smiles):
    mol = dm.to_mol(smiles)  # Convert SMILES to a molecular object

    if mol is None:
        return None

    dm_data = {
        "hbd": dm.descriptors.n_lipinski_hbd(mol),
        "hba": dm.descriptors.n_lipinski_hba(mol),
    }

    dm_df = pd.DataFrame([dm_data])  # Create a DataFrame from the dm_data dictionary

    return dm_df

def create_dm_dataframe(df):
    feature_data = []

    for index, row in df.iterrows():
        smiles = row['SMILES']
        dm_df = calculate_molecular_properties_dm(smiles)

        if dm_df is not None:
            feature_data.append(dm_df)
        combined_df = pd.concat(feature_data, ignore_index=True)
        combined_df['Solubility FI'] = df['LogD'] + Data['NumAromaticRings']
        return combined_df

HBDHBA = create_dm_dataframe(Data)

In [ ]:
HBDHBA.isnull().sum()

hbd              0
hba              0
max_ring_size    0
dtype: int64

In [ ]:
# The Solubility Forecast Index = cLogDpH7.4 + #Ar  published in 2010 by Alan Hill and Robert Young at GSK, provides a simple, elegant equation for estimating aqueous solubility.
HBDHBA['SFI'] = Data['LogD']  + Data['NumAromaticRings']

In [ ]:
HBDHBA

,hbd,hba,max_ring_size,SFI
0,2,7,6,6.259
1,1,4,7,4.172
2,0,7,6,4.585
3,0,8,6,6.475
4,0,4,6,4.337
...,...,...,...,...
5573,1,4,6,5.207
5574,0,7,6,-1.736
5575,1,6,6,3.792
5576,2,5,6,2.423


In [ ]:
# https://colab.research.google.com/github/PatWalters/practical_cheminformatics_tutorials/blob/main/sar_analysis/find_scaffolds.ipynb#scrollTo=e38b7396

In [ ]:
FCFP_8_4096_2[:-483].to_csv('/content/drive/MyDrive/metabolism_dacon/features/FCFP_8_4096_2_train.csv',index=False)
FCFP_8_4096_2[-483:].to_csv('/content/drive/MyDrive/metabolism_dacon/features/FCFP_8_4096_2_test.csv',index=False)

# FCFP_4_2048[:-483].to_csv('/content/drive/MyDrive/metabolism_dacon/features/FCFP_4_2048_train.csv',index=False)
# FCFP_4_2048[-483:].to_csv('/content/drive/MyDrive/metabolism_dacon/features/FCFP_4_2048_test.csv',index=False)

# FCFP_6_4096[:-483].to_csv('/content/drive/MyDrive/metabolism_dacon/features/FCFP_6_4096_train.csv',index=False)
# FCFP_6_4096[-483:].to_csv('/content/drive/MyDrive/metabolism_dacon/features/FCFP_6_4096_test.csv',index=False)

# MACCS_keys[:-483].to_csv('/content/drive/MyDrive/metabolism_dacon/features/MACCS_keys_train.csv',index=False)
# MACCS_keys[-483:].to_csv('/content/drive/MyDrive/metabolism_dacon/features/MACCS_keys_test.csv',index=False)

# ECFP_fp_4096[:-483].to_csv('/content/drive/MyDrive/metabolism_dacon/features/ECFP_2_4096_train.csv',index=False)
# ECFP_fp_4096[-483:].to_csv('/content/drive/MyDrive/metabolism_dacon/features/ECFP_2_4096_test.csv',index=False)

# HBDHBA[:-483].to_csv('/content/drive/MyDrive/metabolism_dacon/features/HBDHBA_sol_train.csv',index=False)
# HBDHBA[-483:].to_csv('/content/drive/MyDrive/metabolism_dacon/features/HBDHBA_sol_test.csv',index=False)

In [ ]:
ring_df[:-483].to_csv('/content/drive/MyDrive/metabolism_dacon/features/ring_counts_train.csv',index=False)
ring_df[-483:].to_csv('/content/drive/MyDrive/metabolism_dacon/features/ring_counts_test.csv',index=False)

HBA, HBB

https://practicalcheminformatics.blogspot.com/2023/06/getting-real-with-molecular-property.html

In [ ]:
특정구조를 갖는지에 대한 feature

In [ ]:
reaction 가능한지

In [ ]:
https://github.com/PatWalters/resources/blob/main/cheminformatics_resources.md

https://github.com/PatWalters/resources/blob/main/cheminformatics_resources.md

In [ ]:
# Rules of CNS
# BBB score
# Rule of Egan
# Rule-of-5
# Beyond Rule-of-5
# Rule-of-4
# Ghose Filter
# Zinc Rule
# Rule of GSK (4/400)
# Lead-Like Soft Rule
# Oprea’s Rule
# Pfizer Rule (3/75)
# REOS Filter
# Rule-of-3
# Extended Rule-of-3
# Veber Filter